# Working Basilica.ai Medicine Cabinet Build Group 3 Model
## With user_text input and top_5_recommendations output


do a calculation-comparison using the stored values to save time, 
instead of both:

A. calculating the user_text_embedding each time, we will only do it once

B calculating the cultivate embedding for each query, we will only do it once

So the query time ~should be reduced to just the comparison calculations.


(maybe switch to spaCy tools later for similar outcome)




larger document of notes and experiments here https://colab.research.google.com/drive/1IWwmn0lRHEgwvw4l4ReAuF7xb0-d8nBC

In [0]:
!pip install basilica

In [0]:
# Import The Libraries & Packages
import basilica
import numpy as np
import pandas as pd
from scipy import spatial

In [0]:
# Set The Variables

user_input = ""

user_input_embedding = []

In [0]:
#so more head displays
#override display option
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
# csv here for download
#https://drive.google.com/open?id=15-KMmSgxISrH8WtGGZPSB-C7DANSMLny

In [0]:
# download the data into the current working directory/folder

#!wget https://raw.githubusercontent.com/MedCabinet/ML_Machine_Learning_Files/master/med1.csv
!wget https://github.com/lineality/4.4_Build_files/raw/master/symptoms8_medcab3.csv


# Website for the current data source: https://www.kaggle.com/kingburrito666/cannabis-strains

--2020-01-09 00:01:26--  https://github.com/lineality/4.4_Build_files/raw/master/symptoms8_medcab3.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lineality/4.4_Build_files/master/symptoms8_medcab3.csv [following]
--2020-01-09 00:01:27--  https://raw.githubusercontent.com/lineality/4.4_Build_files/master/symptoms8_medcab3.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1497841 (1.4M) [text/plain]
Saving to: ‘symptoms8_medcab3.csv.1’

symptoms8_medcab3.c 100%[===================>]   1.43M  --.-KB/s    in 0.1s    

2020-01-09 00:01:29 (11.8 MB/s) - ‘symptoms8_medcab3.csv.1’ saved [1497841/14978

In [0]:
# import the stored embedding file values as a csv

In [0]:
# inspect, see what's loaded
!ls

sample_data	       symptoms8_medcab3.csv
symptommedembedv8.pkl  symptoms8_medcab3.csv.1


In [0]:
# put the data into a dataframe named df
#df = pd.read_csv('med1.csv')

In [0]:
# inspect
#df.head(2)

In [0]:
# see below: words were incorrectly
# compounded around commas, this removes the commas
#df['Description'] = df['Description'].str.replace(',', ' ')
#df['Description'] = df['Description'].str.replace(',', ' ')

In [0]:
# inspect
#df.head(2)

In [0]:
ls

sample_data/           symptoms8_medcab3.csv
symptommedembedv8.pkl  symptoms8_medcab3.csv.1


In [0]:
# for colab
# run this cell, you will be prompted to upload, click on choose file, 
# then drag and drop the file in
# upload file from local drives
import os
import pandas as pd
from google.colab import files

#uploaded = files.upload()
!ls #check file is there
# add that file name to the below and run that
#df = pd.read_csv('file_name')

sample_data	       symptoms8_medcab3.csv
symptommedembedv8.pkl  symptoms8_medcab3.csv.1


In [0]:
# Load updated file 
df = pd.read_csv('symptoms8_medcab3.csv')

# Feature Engineering: 
### creating a single feature-column text-string containing the product description: flavour and effects

### Text Blob:

The valueable text for the customer is the flavor and effects. The description is not medically helpful except in rare cases.

Given that the high cardinality of Y (lots of targets to pick from) may lend itself to a 'search engine' type method, the version of the dataframe below merges all the text features into one column. Commas were added to avoid the text-merging splicing two words into one compounded word. The flavours, effects, and description columns were merged. 

In [0]:
# a space needs to be added after the commas in the Effects and flavor feature-columns
#df['Flavor'] = df['Flavor'].str.replace(',', ', ')
#df['Effects'] = df['Effects'].str.replace(',', ', ')

In [0]:
#Commas were added to avoid the text-merging splicing two words into one compounded word.
#df['Flavor_plus'] = ', ' + df['Flavor'].astype(str)


In [0]:
#df['Effects_and_Flavor'] = df['Effects'].map(str) + df['Flavor_plus']

# Basilica Med3 Fast_Model_v3
https://www.basilica.ai/quickstart/python/

# Entering the User Input

In [0]:
user_input = "text, Relaxed, Violet, Aroused, Creative, Happy, Energetic, Flowery, Diesel"

## a function to calculate_user_text_embedding

In [0]:
# (works)
# a function to calculate_user_text_embedding
# to save the embedding value in session memory

score = 0

def calculate_user_text_embedding(input, user_input_embedding):

  # setting a string of two sentences for the algo to compare
  sentences = [input]

  # calculating embedding for both user_entered_text and for features
  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:
    user_input_embedding = list(c.embed_sentences(sentences))
  
  return user_input_embedding

# run the function to save the embedding value in session memory
user_input_embedding = calculate_user_text_embedding(user_input, user_input_embedding)

## Function for a Loop that makes and stores score value in column


In [0]:
'''
# a function to look for the similarity score, comparing the user input
# to each of the cultivars (strains)

score = 0

def score_the_user_input_by_row(input, score, row1):

  # flavor and effect of a cultivar (a row)
  item1 = df.loc[row1, 'Effects_and_Flavor']

  # setting a string of two sentences for the algo to compare
  sentences = [item1,input]

  # calculating embedding for both user_entered_text and for features
  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:
    embeddings = list(c.embed_sentences(sentences))
  
  #embeddings = np.str(embeddings)
  embeddings = np.array(embeddings)


  # adjusted 'inverse' score so a higher number is 'better'
  score = 1 - spatial.distance.cosine(embeddings[0], embeddings[1])

  # returns a variable that can be used outside of the function
  return score
  '''

"\n# a function to look for the similarity score, comparing the user input\n# to each of the cultivars (strains)\n\nscore = 0\n\ndef score_the_user_input_by_row(input, score, row1):\n\n  # flavor and effect of a cultivar (a row)\n  item1 = df.loc[row1, 'Effects_and_Flavor']\n\n  # setting a string of two sentences for the algo to compare\n  sentences = [item1,input]\n\n  # calculating embedding for both user_entered_text and for features\n  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:\n    embeddings = list(c.embed_sentences(sentences))\n  \n  #embeddings = np.str(embeddings)\n  embeddings = np.array(embeddings)\n\n\n  # adjusted 'inverse' score so a higher number is 'better'\n  score = 1 - spatial.distance.cosine(embeddings[0], embeddings[1])\n\n  # returns a variable that can be used outside of the function\n  return score\n  "

## Function for a Loop that makes and stores score value in column using stored (not calculated) variables (user_input_embedding) to compare and score two strings

In [0]:
'''
# works
# Experimental function using variable: user_input_embedding
# to compare two strings
# spatial.distance.cosine(embeddings[0], user_input_embedding[0])
# last [0] is optional...
# spatial.distance.cosine(embeddings[0], user_input_embedding)

# a function to look for the similarity score, comparing the user input
# to each of the cultivars (strains)

score = 0

def score_the_user_input_by_row(input, score, row1):

  # flavor and effect of a cultivar (a row)
  item1 = df.loc[row1, 'Effects_and_Flavor']

  # setting a string of two sentences for the algo to compare
  sentences = [item1,input]

  # calculating embedding for both user_entered_text and for features
  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:
    embeddings = list(c.embed_sentences(sentences))
  
  #embeddings = np.str(embeddings)
  embeddings = np.array(embeddings)

  # adjusted 'inverse' score so a higher number is 'better'
  score = 1 - spatial.distance.cosine(embeddings[0], user_input_embedding[0])

  # returns a variable that can be used outside of the function
  return score
'''

"\n# works\n# Experimental function using variable: user_input_embedding\n# to compare two strings\n# spatial.distance.cosine(embeddings[0], user_input_embedding[0])\n# last [0] is optional...\n# spatial.distance.cosine(embeddings[0], user_input_embedding)\n\n# a function to look for the similarity score, comparing the user input\n# to each of the cultivars (strains)\n\nscore = 0\n\ndef score_the_user_input_by_row(input, score, row1):\n\n  # flavor and effect of a cultivar (a row)\n  item1 = df.loc[row1, 'Effects_and_Flavor']\n\n  # setting a string of two sentences for the algo to compare\n  sentences = [item1,input]\n\n  # calculating embedding for both user_entered_text and for features\n  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:\n    embeddings = list(c.embed_sentences(sentences))\n  \n  #embeddings = np.str(embeddings)\n  embeddings = np.array(embeddings)\n\n  # adjusted 'inverse' score so a higher number is 'better'\n  score = 1 - spatial.distance.co

In [0]:
df.shape

(2351, 10)

This Loop creates the scores for the final top-5 tally.

In [0]:
'''
# For the real model this will be set to the fill dataset value, currently 2351
# see dataframe shape above

for i in range(5):
  # calls the function to set the value of 'score'
  # which is the score of the user input
  score = score_the_user_input_by_row(user_input, score, i)
  #stores the score in the dataframe
  df.loc[i,'score'] = score
  # optionally prints the score as verbosity for the user
  print(df.loc[i,'score'])
  '''

"\n# For the real model this will be set to the fill dataset value, currently 2351\n# see dataframe shape above\n\nfor i in range(5):\n  # calls the function to set the value of 'score'\n  # which is the score of the user input\n  score = score_the_user_input_by_row(user_input, score, i)\n  #stores the score in the dataframe\n  df.loc[i,'score'] = score\n  # optionally prints the score as verbosity for the user\n  print(df.loc[i,'score'])\n  "

# This is the final output of the top 5 matching predictions

In [0]:
'''
output = df['Strain'].groupby(df['score']).value_counts().nlargest(5, keep='last')
print(output)
'''

"\noutput = df['Strain'].groupby(df['score']).value_counts().nlargest(5, keep='last')\nprint(output)\n"

## A Function for Pre-calculating Embedding_values for each cultivar's description

---



In [0]:
df.head(1)

,Unnamed: 0,Unnamed: 0.1,Strain,Type,Rating,Effects,Flavor,Description,symptoms_diseases,all_text_search
0,0,0,100-Og,hybrid,4.0,"Creative, Energetic, Tingly, Euphoric, Relaxed","Earthy, Sweet, Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"ms, pain,","ms, pain, Creative, Energetic, Tingly, Euphori..."


# For Symptoms Diseases

In [0]:
# a function to calculate the embedding_value of each cultivar feature/column/flavor_and_effects
# for each of the cultivars (strains) / for each row
# this function is run through a for-loop

embeddings = 0

def calculate_embedding_of_feature(embeddings, row1):

  # flavor and effect of a cultivar (a row)
  item1 = df.loc[row1, 'symptoms_diseases']

  # setting a string of two sentences for the algo to compare
  sentences = [item1]

  # calculating embedding for both user_entered_text and for features
  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:
    embeddings = list(c.embed_sentences(sentences))
  
  # adjusted 'inverse' score so a higher number is 'better'
  #embedding1 = 1 - spatial.distance.cosine(embeddings[0])

  # returns a variable that can be used outside of the function
  return embeddings

# For All Search Text

In [0]:
# a function to calculate the embedding_value of each cultivar feature/column/flavor_and_effects
# for each of the cultivars (strains) / for each row
# this function is run through a for-loop

embeddings = 0

def calculate_embedding_of_feature(embeddings, row1):

  # flavor and effect of a cultivar (a row)
  item1 = df.loc[row1, 'all_text_search']

  # setting a string of two sentences for the algo to compare
  sentences = [item1]

  # calculating embedding for both user_entered_text and for features
  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:
    embeddings = list(c.embed_sentences(sentences))
  
  # adjusted 'inverse' score so a higher number is 'better'
  #embedding1 = 1 - spatial.distance.cosine(embeddings[0])

  # returns a variable that can be used outside of the function
  return embeddings

In [0]:
#user_input_embedding = np.squeeze(np.asarray(user_input_embedding))

In [0]:
df.shape

(2351, 10)

In [0]:
#df.head(1)

In [0]:

# v1
# Stores each row's feature_effect_flavor/column embedding in a new colum
# This is s a for loop 
# that uses the "calculate_embedding_of_feature" function

# For the real model this will be set to the fill dataset value, currently 2351
# see dataframe shape above

for i in range(1):

  # calls the function to set the value of 'score'
  # which is the score of the user input
  embeddings = calculate_embedding_of_feature(embeddings, i)
  #stores the score in the dataframe
  
  #this formats the embeddings as a string, which must be turned
  # back into an array for comparison later:
  # with this >>> embeddings = np.array(embeddings)

  #embeddings = np.squeeze(np.asarray(embeddings))
  embeddings = np.str(embeddings)

  df.loc[i,'embedding_of_feature'] = embeddings
  # optionally prints the score as verbosity for the user
  #print(df.loc[i,'embedding_of_feature'])
  print(i)

  

In [0]:
# The problem is here...

# works
#df_embeddings = pd.DataFrame([embeddings])


In [0]:
#df_embeddings.head()

In [0]:
#df_embeddings.shape

In [0]:
# works
#df_embeddings = pd.DataFrame([embeddings])


# To Make Pickle Start Here

## For-Loop that sets up a Dataframe in which cultivar embeddings are stored

In [0]:
# works
for i in range(1):
  embeddings = calculate_embedding_of_feature(embeddings, i)
  df_embeddings = pd.DataFrame([embeddings])

In [0]:
df_embeddings

## Stores each row's feature_effect_flavor/column embedding in a new datafram's new column
This uses a 2nd database to store the values (to avoid errrors)
which must (to avoid errors) be created using a sample array

ideally some other method should be used to create a Dataframe

In [0]:
df.shape

In [0]:
df.isna().sum()

In [0]:
df.head(2)

In [0]:
df_embeddings

In [0]:
#v2

#37 minutes to run all 2351 rows

# https://duckduckgo.com/?q=add+value+as+new+row+to+a+dataframe+pandas&t=ffnt&atb=v200-3&ia=web&iax=qa

# Stores each row's feature_effect_flavor/column embedding in a new colum
# This is for a for-loop 
# that uses the "calculate_embedding_of_feature" function

# For the real model this will be set to the fill dataset value, currently 2351
# see dataframe shape above

for i in range(2351):

  # calls the function to calculate_embedding_of_feature
  embeddings = calculate_embedding_of_feature(embeddings, i)

  #stores the score in the dataframe
  df_embeddings.loc[i] = [embeddings]

  # optionally prints as verbosity for the user
  #print(df_embeddings.loc[i])
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
df_embeddings.head(1)

,0
0,"[[-0.225251, 0.248883, -0.00214154, -0.0111227..."


Saving CSV to file

In [0]:
#exports df (current working directory)
#df_embeddings.to_csv('med3_symptom_embeddings_v1.csv')

# Saving encoding as Pickle File

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_pickle.html

In [0]:
df_embeddings.to_pickle("./symptommedembedv8.pkl")

In [0]:
# check to see file is there
ls

sample_data/           symptoms8_medcab3.csv
symptommedembedv8.pkl  symptoms8_medcab3.csv.1


In [0]:
#unpickling file
unpickled_df_test = pd.read_pickle("./symptommedembedv8.pkl")

In [0]:
unpickled_df_test.head(2)

,0
0,"[[-0.225251, 0.248883, -0.00214154, -0.0111227..."
1,"[[-1.28391, -0.538323, -0.0280458, -0.362102, ..."


# Downloading Files from Colab

In [0]:
from google.colab import files
#files.download('med3_embeddings_v1.csv')

In [0]:
files.download('symptommedembedv8.pkl')

# Loading embeddings CSV => Dataframe from file

In [0]:
# imports stored embeddings as CSV and (re)stores as dataframe
#df_embeddings_test = pd.read_csv('med3_embeddings_v1.csv')
#df_embeddings_test = df_embeddings_test.drop(['Unnamed: 0'], axis=1)

In [0]:
#df_embeddings_test.columns()

In [0]:
#df_embeddings_test.head()

,0
0,"[[-0.321601, -0.316229, 0.279052, 0.00351129, ..."
1,"[[-0.124443, -0.182427, 0.313895, 0.260361, -0..."
2,"[[-0.303236, -0.188614, 0.406775, 0.0149814, -..."
3,"[[-0.303083, -0.232994, 0.156894, -0.105752, -..."
4,"[[-0.472175, -0.305105, 0.240355, -0.156201, -..."


# To Do!

# remove unnamed column
# remove the first entry (duplicate)

In [0]:
# remove unnamed column
# remove the first entry (duplicate)

In [0]:
'''
# a function to look for the similarity score, comparing the user input
# to each of the cultivars (strains)

score = 0

def score_the_user_input_by_row(input, score, row1):

  # flavor and effect of a cultivar (a row)
  item1 = df.loc[row1, 'Effects_and_Flavor']

  # setting a string of two sentences for the algo to compare
  sentences = [item1,input]

  # calculating embedding for both user_entered_text and for features
  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:
    embeddings = list(c.embed_sentences(sentences))
  
  # adjusted 'inverse' score so a higher number is 'better'
  score = embeddings

  # returns a variable that can be used outside of the function
  return score
  '''

"\n# a function to look for the similarity score, comparing the user input\n# to each of the cultivars (strains)\n\nscore = 0\n\ndef score_the_user_input_by_row(input, score, row1):\n\n  # flavor and effect of a cultivar (a row)\n  item1 = df.loc[row1, 'Effects_and_Flavor']\n\n  # setting a string of two sentences for the algo to compare\n  sentences = [item1,input]\n\n  # calculating embedding for both user_entered_text and for features\n  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:\n    embeddings = list(c.embed_sentences(sentences))\n  \n  # adjusted 'inverse' score so a higher number is 'better'\n  score = embeddings\n\n  # returns a variable that can be used outside of the function\n  return score\n  "

In [0]:
'''
# To save data as a column/feature, it must be stored as a string. 
# https://stackoverflow.com/questions/50023635/python-convert-string-to-an-array

# you can go back and forth between these forms without loosing data

embeddings = np.array(embeddings)
embeddings.shape
type(embeddings)

embeddings = np.str(embeddings)
type(embeddings)
'''

'\n# To save data as a column/feature, it must be stored as a string. \n# https://stackoverflow.com/questions/50023635/python-convert-string-to-an-array\n\n# you can go back and forth between these forms without loosing data\n\nembeddings = np.array(embeddings)\nembeddings.shape\ntype(embeddings)\n\nembeddings = np.str(embeddings)\ntype(embeddings)\n'

In [0]:
'''
# used above
# (works)
# a function to calculate_user_text_embedding
# to save the embedding value in session memory

score = 0

def calculate_user_text_embedding(input, user_input_embedding):

  # setting a string of two sentences for the algo to compare
  sentences = [input]

  # calculating embedding for both user_entered_text and for features
  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:
    user_input_embedding = list(c.embed_sentences(sentences))
  
  return user_input_embedding

# run the function to save the embedding value in session memory
user_input_embedding = calculate_user_text_embedding(user_input, user_input_embedding)
'''

"\n# used above\n# (works)\n# a function to calculate_user_text_embedding\n# to save the embedding value in session memory\n\nscore = 0\n\ndef calculate_user_text_embedding(input, user_input_embedding):\n\n  # setting a string of two sentences for the algo to compare\n  sentences = [input]\n\n  # calculating embedding for both user_entered_text and for features\n  with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:\n    user_input_embedding = list(c.embed_sentences(sentences))\n  \n  return user_input_embedding\n\n# run the function to save the embedding value in session memory\nuser_input_embedding = calculate_user_text_embedding(user_input, user_input_embedding)\n"

In [0]:
# inspecting, print
#user_input_embedding

In [0]:
df.head(1)

,Strain,Type,Rating,Effects,Flavor,Description,Flavor_plus,Effects_and_Flavor
0,100-Og,hybrid,4.0,"Creative, Energetic, Tingly, Euphoric, Relaxed","Earthy, Sweet, Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,", Earthy, Sweet, Citrus","Creative, Energetic, Tingly, Euphoric, Relaxed..."


In [0]:
'''
# score_user_input_from_stored_embedding_from_stored_values

# string filtering is needed to separate the added garbage characters
# that were introduced when the array was stored as a string

# a function to look for the similarity score, comparing the user input
# to each of the cultivars (strains)

score = 0

# add more parameters?
def score_user_input_from_stored_embedding_from_stored_values(input, score, row1, user_input_embedding):

  # flavor and effect of a cultivar (a row)
  #item1 = df.loc[row1, 'Effects_and_Flavor']

  # setting a string of two sentences for the algo to compare
  #sentences = [input]

  # calculating embedding for both user_entered_text and for features
  #with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:
  #  embedding_new = list(c.embed_sentences(sentences))

  embedding_stored = df.loc[row1, 'embedding_of_feature']
  #print(embedding_stored)

  #embeddings = np.array(embeddings)
  #embeddings = embeddings[0]

  embedding_stored = np.array(embedding_stored)
  embedding_stored = embedding_stored[0]

  print(type(embedding_stored))
  print (len(embedding_stored))

  #embedding_stored = embedding_stored[1:]
  #print(embedding_stored)

  #embedding_stored = np.str(embedding_stored)
  #embedding_stored = np.array(embedding_stored)
  #user_input_embedding = np.array(user_input_embedding, dtype='float64')

  #user_input_embedding = np.squeeze(np.asarray(user_input_embedding))
  #embedding_stored = np.squeeze(np.asarray(embedding_stored))

  #merged = [[embedding_stored],[user_input_embedding]]
  #print(merged)
  #merged = [embedding_stored]

  # adjusted 'inverse' score so a higher number is 'better'
  score = 1 - spatial.distance.cosine(embedding_stored, user_input_embedding)

  # returns a variable that can be used outside of the function
  return score
  '''

"\n# score_user_input_from_stored_embedding_from_stored_values\n\n# string filtering is needed to separate the added garbage characters\n# that were introduced when the array was stored as a string\n\n# a function to look for the similarity score, comparing the user input\n# to each of the cultivars (strains)\n\nscore = 0\n\n# add more parameters?\ndef score_user_input_from_stored_embedding_from_stored_values(input, score, row1, user_input_embedding):\n\n  # flavor and effect of a cultivar (a row)\n  #item1 = df.loc[row1, 'Effects_and_Flavor']\n\n  # setting a string of two sentences for the algo to compare\n  #sentences = [input]\n\n  # calculating embedding for both user_entered_text and for features\n  #with basilica.Connection('36a370e3-becb-99f5-93a0-a92344e78eab') as c:\n  #  embedding_new = list(c.embed_sentences(sentences))\n\n  embedding_stored = df.loc[row1, 'embedding_of_feature']\n  #print(embedding_stored)\n\n  #embeddings = np.array(embeddings)\n  #embeddings = embeddings

In [0]:
df_embeddings.head(2)

,0
0,"[[-0.321601, -0.316229, 0.279052, 0.00351129, ..."
1,"[[-0.124443, -0.182427, 0.313895, 0.260361, -0..."


In [0]:
df_embeddings_test.head(2)

,0
0,"[[-0.321601, -0.316229, 0.279052, 0.00351129, ..."
1,"[[-0.124443, -0.182427, 0.313895, 0.260361, -0..."


In [0]:
df_embeddings_test

,0
0,"[[-0.321601, -0.316229, 0.279052, 0.00351129, ..."
1,"[[-0.124443, -0.182427, 0.313895, 0.260361, -0..."
2,"[[-0.303236, -0.188614, 0.406775, 0.0149814, -..."
3,"[[-0.303083, -0.232994, 0.156894, -0.105752, -..."
4,"[[-0.472175, -0.305105, 0.240355, -0.156201, -..."
5,"[[-0.0817752, -0.0773705, 0.328175, 0.196452, ..."
6,"[[-0.549368, -0.278196, 0.359297, -0.114844, -..."
7,"[[-0.409113, -0.272425, 0.430481, -0.067206, -..."
8,"[[-0.603517, -0.22713, 0.277611, 0.0888017, -0..."
9,"[[-0.404744, -0.197039, 0.357332, -0.0584108, ..."


In [0]:
type(df_embeddings.loc[1, 0])

list

# Function to score_user_input_from_stored_embedding_from_stored_values
## goes with a for-loop

In [0]:
# (works)
#v2
# score_user_input_from_stored_embedding_from_stored_values
# a function to look for the similarity score, comparing the user input
# to each of the cultivars (strains)

score = 0

def score_user_input_from_stored_embedding_from_stored_values(input, score, row1, user_input_embedding):

  #nope
  #embedding_stored = df_embeddings_test.loc[row1, 0]

  # experiment
  embedding_stored = unpickled_df_test.loc[row1, 0]
  
  # works: original
  #embedding_stored = df_embeddings.loc[row1, 0]

  score = 1 - spatial.distance.cosine(embedding_stored, user_input_embedding)

  # returns a variable that can be used outside of the function
  return score


# For-Loop to score_user_input_from_stored_embedding_from_stored_values

In [0]:

# For the real model this will be set to the fill dataset value, currently 2351
# see dataframe shape above

for i in range(2351):
  # calls the function to set the value of 'score'
  # which is the score of the user input
  score = score_user_input_from_stored_embedding_from_stored_values(user_input, score, i, user_input_embedding)
  #stores the score in the dataframe
  df.loc[i,'score'] = score
  # optionally prints the score as verbosity for the user
  print(df.loc[i,'score'])
  print(i)

0.8958629380898788
0
0.9835731747824231
1
0.9267263906572931
2
0.8898603617876185
3
0.9143115221493963
4
0.8529647922908515
5
0.8755999385082817
6
0.8955610022961553
7
0.908232820950232
8
0.92935373487371
9
0.9530002546630619
10
0.9136784618120394
11
0.942677246285248
12
0.879898067918814
13
0.9269940557671099
14
0.9261556379649547
15
0.8529647922908515
16
0.9120787817371911
17
0.9004727746604987
18
0.9226266400220631
19
0.9295822126706159
20
0.8900693718473759
21
0.9374186748204477
22
0.8850608592854549
23
0.8813553241650606
24
0.933629334608168
25
0.9544040284660802
26
0.8573337221501854
27
0.9015408027442589
28
0.9217640444583428
29
0.9128371373421914
30
0.8920121724911799
31
0.9109340885343756
32
0.8773530932600087
33
0.9090966979339873
34
0.9364905747773015
35
0.9383054242090338
36
0.8756584955795639
37
0.9442600194768948
38
0.9225286187526353
39
0.9302899370508412
40
0.8992971801225401
41
0.9136825173592534
42
0.935581865851087
43
0.9436431528335192
44
0.9055137007917144
45
0.922

In [0]:
output = df['Strain'].groupby(df['score']).value_counts().nlargest(5, keep='last')

In [0]:
output

score     Strain        
0.905756  B-Witched         2
0.983573  98-White-Widow    1
0.969837  Purple-Diesel     1
0.964103  Blue-Monster      1
0.962950  Deadhead-Og       1
Name: Strain, dtype: int64